<a href="https://colab.research.google.com/github/JTumelty/Smart-Health/blob/main/Project-3/MLSH_L3_Project_JTumelty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instructor: Juber Rahman<br>
Dataset: St Petersburg INCART dataset from Physionet<br>
Signal: 12-lead ECG<br>
Pre-processing: neurokit<br>
Method: 1-D CNN or Multi-head CNN<br>
last updated: Nov 12, 2021<br>

# Project 3
##Instructions:
###Prework:
1.  Watch a 12 lead ECG video https://www.youtube.com/watch?v=kwLbSx9BNbU 

> **Notes:** 
>
>12 leads of the ECG represent the electrical signal viewed from 12 angles
  * Limb leads look at the heart in a vertical plane and obtained from three electrodes (LA,RA, LL). 
    * Lead 1: Voltage between right arm and left arm (look at heart from left).
    * Lead 2: Voltage between right arm and left leg (looks at heart from inferior left)
    * Lead 3: Voltage between the left arm and the left leg (looks at heart from inferior right)
    * aVR: unipolar (looks at upper right)
    * aVL: unipolar (looks at upper left)
    * aVF: unipolar (looks at inferior wall)
  * Chest leads: view the heart in a horizontal plane and are unipolar leads 
    * Depolarisation toward a lead: positive
    * Depolarisation away from a lead: negative



2.  Select a 12 lead ECG database (e.g. St. 
Petersburg INCART Database) from Physionet and read the dataset descriptions to identify the heart patient types (e.g MI, CAD, TIA, Other)

> **Notes:**
> We use the St Petersburg INCART 12-lead Arrhythmia Database (https://doi.org/10.13026/C2V88N), where records have the following heart diseases.
* **Acute MI---2**
* **Transient ischemic attack (angina pectoris)---5**
* **Prior MI---4**
* **Coronary artery disease with hypertension---7 (4 with ECGs consistent with left ventricular hypertrophy)**
* Sinus node dysfunction---1
* Supraventricular ectopy---18
* Atrial fibrillation or SVTA---3 (2 with paroxysmal AF)
* WPW---2
* AV block---1
* Bundle branch block---3
>
> We have marked in bold the four conditions that we are particularly interested in classifying. 

3. Have a look at the project 3 starter notebook
load and process the dataset using python WFDB, neurokit, biosppy etc. packages
read about the different ECG channels and pick the best channel signal e.g. V2, V5 etc. 
Divide each ECG record into 1 minute chunks so in total you have 32 x no of hours x 60 ~ 23000 samples 

> **Notes:**
>
> From https://doi.org/10.1016/j.cmpb.2015.12.008, indicates lead II is frequently used in detecting heart diseases. Alternatively, lead V allows for classification of ventricular related arrhythmias. In this project, we will therefore consider using lead 2 and lead 5 separately.
> From the summary data: https://physionet.org/lightwave/?db=incartdb/1.0.0, we find that V2 corresponds to channel 7 and V5 corresponds to channel 10.

4. How these diseases MI, CAD, TIA etc. differ from each other, what morphological changes are introduced in the ECG signal? 

> **Notes:**
> * MI: https://www.ahajournals.org/doi/epub/10.1161/CIR.0b013e31826e1058
>   * Acute or evolving
changes in the ST-T waveforms and Q waves. 
>   * New, or presumed new, J point elevation ⱖ0.1 mV is
required in all leads other than V 2 and V 3. Note that there is a difference between men and women.
> * CAD: **TO DO**
> * TIA: **TO DO**
>
> Time permitting, we will later try to determine whether these properties agree with feature identification of the neural networks.

5. Develop an articial neural network for multi-category heart disease classification from single channel ECG
6. Develop a 1-D convolutional neural network for multi-category heart disease classification from single channel ECG
7. Compare the performances of the developed deep learning models



## Import packages and download data

In [3]:
!pip install wfdb
!pip install peakutils
!pip install neurokit2
import neurokit2 as nk
import wfdb
from wfdb import processing
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import peakutils

Download all the WFDB records and annotations from a small Physionet Database: INCARTDB

In [ ]:
# Make a temporary download directory in your current working directory
cwd = os.getcwd()
dl_dir = os.path.join(cwd, 'tmp_dl_dir')

# Download all the WFDB content
wfdb.dl_database('incartdb', dl_dir=dl_dir)

# Display the downloaded content in the folder
display(os.listdir(dl_dir))

# Cleanup: delete the downloaded directory
#shutil.rmtree(dl_dir)

Generating record list for: I01
Generating record list for: I02
Generating record list for: I03
Generating record list for: I04
Generating record list for: I05
Generating record list for: I06
Generating record list for: I07
Generating record list for: I08
Generating record list for: I09
Generating record list for: I10
Generating record list for: I11
Generating record list for: I12
Generating record list for: I13
Generating record list for: I14
Generating record list for: I15
Generating record list for: I16
Generating record list for: I17
Generating record list for: I18
Generating record list for: I19
Generating record list for: I20
Generating record list for: I21
Generating record list for: I22
Generating record list for: I23
Generating record list for: I24
Generating record list for: I25
Generating record list for: I26
Generating record list for: I27
Generating record list for: I28
Generating record list for: I29
Generating record list for: I30
Generating record list for: I31
Generati

['I44.atr',
 'I42.hea',
 'I62.atr',
 'I29.dat',
 'I17.atr',
 'I75.hea',
 'I38.atr',
 'I34.hea',
 'I59.atr',
 'I57.atr',
 'I02.atr',
 'I14.atr',
 'I10.dat',
 'I30.atr',
 'I29.hea',
 'I46.atr',
 'I02.dat',
 'I61.atr',
 'I22.atr',
 'I21.dat',
 'I50.dat',
 'I22.dat',
 'I40.hea',
 'I01.hea',
 'I03.atr',
 'I36.dat',
 'I47.atr',
 'I24.hea',
 'I71.atr',
 'I31.hea',
 'I32.dat',
 'I24.dat',
 'I35.dat',
 'I55.dat',
 'I06.atr',
 'I01.dat',
 'I60.atr',
 'I74.hea',
 'I51.atr',
 'I33.dat',
 'I73.atr',
 'I11.atr',
 'I53.atr',
 'I59.dat',
 'I07.atr',
 'I26.atr',
 'I09.dat',
 'I69.atr',
 'I73.hea',
 'I67.atr',
 'I37.dat',
 'I08.dat',
 'I16.dat',
 'I67.hea',
 'I69.dat',
 'I56.atr',
 'I55.atr',
 'I11.dat',
 'I64.atr',
 'I11.hea',
 'I71.hea',
 'I15.atr',
 'I66.atr',
 'I61.hea',
 'I46.hea',
 'I65.hea',
 'I13.atr',
 'I49.hea',
 'I70.dat',
 'I63.atr',
 'I15.hea',
 'I38.hea',
 'I68.dat',
 'I48.dat',
 'I40.dat',
 'I74.dat',
 'I72.atr',
 'I31.atr',
 'I65.dat',
 'I30.dat',
 'I53.hea',
 'I60.hea',
 'I47.dat',
 'I6

## Preprocessing for Neural Networks
Using channel V2 as a starting point, we will read in all of the records, create epochs of 60s and make a dataframe combining this information with the disease classification.

In [ ]:
# Find the list of records to consider
rec_list = sorted(list(set([os.path.splitext(x)[0] for x in os.listdir(dl_dir)])))
print(rec_list)

['I01', 'I02', 'I03', 'I04', 'I05', 'I06', 'I07', 'I08', 'I09', 'I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'I16', 'I17', 'I18', 'I19', 'I20', 'I21', 'I22', 'I23', 'I24', 'I25', 'I26', 'I27', 'I28', 'I29', 'I30', 'I31', 'I32', 'I33', 'I34', 'I35', 'I36', 'I37', 'I38', 'I39', 'I40', 'I41', 'I42', 'I43', 'I44', 'I45', 'I46', 'I47', 'I48', 'I49', 'I50', 'I51', 'I52', 'I53', 'I54', 'I55', 'I56', 'I57', 'I58', 'I59', 'I60', 'I61', 'I62', 'I63', 'I64', 'I65', 'I66', 'I67', 'I68', 'I69', 'I70', 'I71', 'I72', 'I73', 'I74', 'I75']


In [ ]:
def patient_info(rec_name,comments):
  '''Classify patient disease:
  Coronary artery disease: 0
  MI: 1
  Transient ischemic attack: 2
  Other: 3
  
  Extract gender: M/F
  Extract age
  '''

  # Disease
  if 'Coronary artery disease' in comments[0]:
    disease = 0
  elif 'MI' in comments[0]:
    disease = 1
  elif 'Transient ischemic attack' in comments[0]:
    disease = 2
  else:
    disease = 3

  # Gender
  if '<sex>: M' in comments[0]:
    gender = 'M'
  else:
    gender = 'F'

  # Age
  age = int(fields['comments'][0][7:9])

  return disease,age,gender

In [ ]:
# Iterate over all patient records and use cleaned ECG data 
# We further only consider 10 minutes out of the 30 minute records to make the analysis more manageable.
for rec_name in rec_list:
  lst = []
  # Read in ECG record for channel V2
  signals_7, fields = wfdb.rdsamp('/content/tmp_dl_dir/'+rec_name, channels=[7],sampto = 257*10*60+1)
  # Extract patient info
  disease,age,gender = patient_info(rec_name,fields['comments'])
  # Process ECG signal
  signals, info = nk.ecg_process(signals_7.flatten() , sampling_rate=257)
  # Create epochs with 60 second intervals
  epochs = nk.epochs_create(signals, sampling_rate=257, epochs_end=60)
  # Iterate over epochs and append clean ecg signal with disease classification to dataframe
  for e_key in epochs.keys():
    ecg_clean = epochs[e_key].T.iloc[1].values.tolist()
    lst.append(ecg_clean+[disease])
    # Add data to datafame
  all_data = pd.DataFrame(lst)
  all_data.to_csv(rec_name+'_split.csv')

  print(rec_name,disease)


I71 1
I72 0
I73 0
I74 1
I75 1


Load in all csv files and split into test and training data

In [ ]:
rec_list = sorted([x for x in os.listdir('.') if x[-3:]=='csv' and x[0] == 'I'])
all_records = pd.read_csv(rec_list[0])
for rec_name in rec_list[1:]:
  rec_join = pd.read_csv(rec_name)
  all_records = pd.concat([all_records,rec_join],ignore_index=True)
  print(rec_name, all_records.shape)

I02_split.csv (20, 15422)
I03_split.csv (30, 15422)
I04_split.csv (40, 15422)
I05_split.csv (50, 15422)
I06_split.csv (60, 15422)
I07_split.csv (70, 15422)
I08_split.csv (80, 15422)
I09_split.csv (90, 15422)
I10_split.csv (100, 15422)
I11_split.csv (110, 15422)
I12_split.csv (120, 15422)
I13_split.csv (130, 15422)
I14_split.csv (140, 15422)
I15_split.csv (150, 15422)
I16_split.csv (160, 15422)
I17_split.csv (170, 15422)
I18_split.csv (180, 15422)
I19_split.csv (190, 15422)
I20_split.csv (200, 15422)
I21_split.csv (210, 15422)
I22_split.csv (220, 15422)
I23_split.csv (230, 15422)
I24_split.csv (240, 15422)
I25_split.csv (250, 15422)
I26_split.csv (260, 15422)
I27_split.csv (270, 15422)
I28_split.csv (280, 15422)
I29_split.csv (290, 15422)
I30_split.csv (300, 15422)
I31_split.csv (310, 15422)
I32_split.csv (320, 15422)
I33_split.csv (330, 15422)
I34_split.csv (340, 15422)
I35_split.csv (350, 15422)
I36_split.csv (360, 15422)
I37_split.csv (370, 15422)
I38_split.csv (380, 15422)
I39_split

In [ ]:
col_drop = all_records.columns[[0,-1]].values

In [4]:
from sklearn.model_selection import train_test_split

In [38]:
X = all_records.drop(col_drop, axis=1)
y = all_records[col_drop[1]]

X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.25, random_state=10 )

Save this dataframe to a csv file so they do not have to be regenerated each time.

In [39]:
X_train.to_csv('X_train.csv')
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')

Check distribution of the classifications

In [40]:
y_train_count = {}
y_test_count = {}
for i in range(4):
  y_train_count[i] = y_train[y_train.values == i].count()
  y_test_count[i] = y_test[y_test.values == i].count()
print('Training data:',y_train_count)
print('Testing data:',y_test_count)


Training data: {0: 128, 1: 112, 2: 68, 3: 247}
Testing data: {0: 42, 1: 38, 2: 22, 3: 83}


Hence we see that the test and training have similar stratifications (as would be expected), but that the categories are unbalanced.

In [56]:
# Read in train and test data
X_train= pd.read_csv('X_train.csv')
X_test= pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')
y_test= pd.read_csv('y_test.csv')

In [57]:
# Drop 'Unnamed: 0' column
unnamed='Unnamed: 0'
X_train = X_train.drop(unnamed, axis=1)
X_test = X_test.drop(unnamed, axis=1)
y_train = y_train.drop(unnamed, axis=1)
y_test = y_test.drop(unnamed, axis=1)


## Artificial Neural Network
Following code adapted from lab 3 notebook

In [13]:
from sklearn.datasets import make_classification
from sklearn.metrics import f1_score
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold, cross_val_score, KFold, StratifiedKFold

In [300]:
# class weighted neural network on an imbalanced classification dataset


# define the neural network model
def define_model(input_dim=None):
	# define model
	model = Sequential()
	# define first hidden layer and visible layer
	model.add(Dense(10, input_dim=input_dim, activation='relu', kernel_initializer='he_uniform'))
	# define output layer: note that because we are interested in a multi-class classification,
	# we dhould change the number of neurons in the final layer to 4: number of categories.
	# define loss and optimizer, which we have changed from binary_crossentropy to categorical_crossentropy
	model.add(Dense(4, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
	return model

def disp_cv_score(X_train,Y_train,epoch_no=100):
	'''Cross validation with 5-fold stratification, evaluating performance using f1 score'''
	n_input = X_train.shape[1]
	cv = StratifiedKFold(n_splits=5, shuffle=False) 
	cv_score_total = 0
	kf = 0
	for train_index, test_index in cv.split(X_train,Y_train):
		model = define_model(n_input)
		X1_train, X1_test = X_train.iloc[train_index], X_train.iloc[test_index]
		Y1_train, Y1_test = Y_train.iloc[train_index], Y_train.iloc[test_index]

		# Convert labels data using encoder to fit the model
		y1_train_vec = np_utils.to_categorical(Y1_train)
		model.fit(X1_train, y1_train_vec,epochs=epoch_no,verbose=0)
	
		# Predict labels on test data
		y_pred = model.predict(X1_test)
		# Convert to single vector
		yclass = argmax(y_pred,axis = 1)
	
		score = f1_score(Y1_test, yclass, average='macro')
		cv_score_total += score
		kf += 1
		print('KFold ',kf,' F1 score: ', score,'\n')
		print(confusion_matrix(Y1_test['15420'].values, yclass))

	print('Average F1 score: ',cv_score_total/5)

In [220]:
# Performance of model on KFold stratification
disp_cv_score(X_train,y_train)

KFold  1  F1 score:  0.43172541743970316 

[[12  3  2  8]
 [ 6  6  4  6]
 [ 2  1  6  5]
 [11  3  7 29]]
KFold  2  F1 score:  0.3111007330559319 

[[ 8  5  5  7]
 [ 6  4  3  9]
 [ 0  2  7  5]
 [16 13  5 16]]
KFold  3  F1 score:  0.27234589396012104 

[[ 5 11  2  8]
 [ 5  7  2  8]
 [ 3  2  4  5]
 [12 13  8 16]]
KFold  4  F1 score:  0.40334012392835916 

[[ 8  7  4  7]
 [ 2  6  2 13]
 [ 3  3  6  1]
 [ 5 12  3 29]]
KFold  5  F1 score:  0.323865961166493 

[[ 7  6  4  9]
 [ 8  4  1 10]
 [ 4  3  5  1]
 [ 7  8  9 25]]
Average F1 score:  0.3484756259101217


We see that the baseline model with one layer with 10 nodes has poor performance with wide variation. Fit the model on the full training data and display confusion matrix when predicted on the test data to understand where the performance issues come from.

In [203]:
model = define_model(n_input)
history = model.fit(X_train, y_train_vec, epochs=100, verbose=0)
yhat = model.predict(X_test)
from numpy import argmax
yclass = argmax(yhat,axis=1)
from sklearn.metrics import confusion_matrix
score = f1_score(y_test, yclass, average='macro')
print('F1 score:',score)
print(confusion_matrix(y_test['15420'].values, yclass))

F1 score: 0.38022568287808134
[[12  9  4 17]
 [ 7 11  8 12]
 [ 5  4  7  6]
 [13 10  7 53]]


Hence, we see that signals are classified fairly uniformly. Two further methods could improve the accuracy of the model:
*   Rebalance the training data
*   Vary the number of layers and the number of nodes in each layer
In the following, we will consider the former.

In [58]:
# Rebalancing the training data by upsampling the minority classes
from sklearn.utils import resample
from sklearn.utils import shuffle

all_data = X_train
all_data['label']=y_train

df_0=all_data.iloc[y_train.index[y_train['15420'] == 0].tolist()]
df_1=all_data.iloc[y_train.index[y_train['15420'] == 1].tolist()]
df_2=all_data.iloc[y_train.index[y_train['15420'] == 2].tolist()]
df_3=all_data.iloc[y_train.index[y_train['15420'] == 3].tolist()]
df_1_upsample=resample(df_1,replace=True,n_samples=250,random_state=123)
df_2_upsample=resample(df_2,replace=True,n_samples=250,random_state=124)
df_0_upsample=resample(df_0,replace=True,n_samples=250,random_state=125)
df_3_upsample=resample(df_3,replace=True,n_samples=250,random_state=123)

train_df_resampled=pd.concat([df_0_upsample,df_1_upsample,df_2_upsample,df_3])

#Shuffle data

train_df_resampled = shuffle(train_df_resampled, random_state=12)

# Split into signal and label
y_rebalance = pd.DataFrame(train_df_resampled['label'])
y_rebalance = y_rebalance.rename(columns = {'label':'15420'})
train_df_resampled = train_df_resampled.drop(['label'],axis=1)


In [329]:
# Apply the model on the resampled training data
disp_cv_score(train_df_resampled,y_rebalance,epoch_no=100)

KFold  1  F1 score:  0.7162369459349022 

[[33  8  0  9]
 [ 2 45  0  3]
 [ 0  0 50  0]
 [12 11  8 19]]
KFold  2  F1 score:  0.8121323391560655 

[[45  1  2  2]
 [ 3 42  0  5]
 [ 0  0 49  1]
 [ 5 11  6 28]]
KFold  3  F1 score:  0.7809157163680013 

[[46  0  2  2]
 [ 3 42  2  3]
 [ 0  0 50  0]
 [10 15  3 21]]
KFold  4  F1 score:  0.7666398010803396 

[[42  2  2  4]
 [ 5 44  0  1]
 [ 1  0 49  0]
 [ 9 11  8 21]]
KFold  5  F1 score:  0.7411184968137119 

[[37  3  4  6]
 [ 4 40  4  2]
 [ 0  0 50  0]
 [12  5  9 23]]
Average F1 score:  0.7634086598706041


Hence we see that performance has noticeably improved by balancing the classes. However, we notice that the final row indicates that normal heart beats are misclassified as belonging to those with a disease. This is potentially because each sample has only 60s worth of ecg signal and the abnormality may not be observed.

In [330]:
model = define_model(n_input)
y_train_vec = np_utils.to_categorical(y_rebalance)
history = model.fit(train_df_resampled,y_train_vec, epochs=65, verbose=0)
yhat = model.predict(X_test)
from numpy import argmax
yclass = argmax(yhat,axis=1)
from sklearn.metrics import confusion_matrix
score = f1_score(y_test, yclass, average='macro')
print('F1 score:',score)
print(confusion_matrix(y_test['15420'].values, yclass))

F1 score: 0.39390954103491627
[[10  5  5 22]
 [ 6 12  6 14]
 [ 7  3  7  5]
 [ 6 13  5 59]]


**This accuracy is low given the performance on each stratified Fold, but I can't work out where the error comes from. Likely that it arises from high n_samples particularly if repeated samples lie between a fold and testing data. This indicates that better performance could be obtained using the full dataset instead of a sample and oversampling.**

# Convolutional Neural Network
We will now attempt this multi-class classification using the convolutional neural network model provided in the lab notebook. 

In [7]:
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [59]:
def network(X_train,y_train,X_test,y_test):
    
    im_shape=(X_train.shape[1],X_train.shape[2])

    inputs_cnn=Input(shape=(im_shape), name='inputs_cnn')
    conv1_1=Convolution1D(64, (6), activation='relu',input_shape=(im_shape))(inputs_cnn)
    # Hidden layer as for ANN
    #conv1_1=BatchNormalization()(conv1_1)
    pool1=MaxPool1D(pool_size=(3), strides=(2), padding="same")(conv1_1)

    flatten=Flatten()(pool1)
    # This reduces the dimensionality of the data, replacing groups by maximal value

 #   conv2_1=Convolution1D(64, (3), activation='relu', input_shape=(im_shape))(pool1)
 #   conv2_1=BatchNormalization()(conv2_1)
 #   pool2=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv2_1)

#    conv3_1=Convolution1D(64, (3), activation='relu', input_shape=(im_shape))(pool2)
#    conv3_1=BatchNormalization()(conv3_1)
#    pool3=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv3_1)

#    flatten=Flatten()(pool3)
    
    dense_end1 = Dense(64, activation='relu')(flatten)
    dense_end1 = Dropout(0.5)(dense_end1)
    dense_end2 = Dense(32, activation='relu')(dense_end1)
    dense_end2 = Dropout(0.5)(dense_end2)
    
    main_output = Dense(4, activation='softmax', name='main_output')(dense_end2)
    # Softmax returns the output probabilities for each class
    
    model = Model(inputs= inputs_cnn, outputs=main_output)
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy'])
    model.summary()
    
    callbacks = [EarlyStopping(monitor='val_loss', mode='min',patience=8)]

    y_train_vec = np_utils.to_categorical(y_train)
    history=model.fit(X_train, y_train_vec,epochs=10,batch_size=32,callbacks=callbacks)

    return(model,history)

Initial exploration indicated that without balancing the dataset, we find that the model tried to classify each result as the majority class, so we will attempt to use the rebalanced training data to fir the model, to see if this provides an improvement in perfomance.

In [60]:
# Resize the testing and training data for use in CNN
train_df_resampled = np.expand_dims(train_df_resampled, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [61]:
model,history=network(train_df_resampled,y_rebalance,X_test,y_test)

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs_cnn (InputLayer)     [(None, 15420, 1)]        0         
                                                                 
 conv1d_42 (Conv1D)          (None, 15415, 64)         448       
                                                                 
 max_pooling1d_42 (MaxPoolin  (None, 7708, 64)         0         
 g1D)                                                            
                                                                 
 flatten_16 (Flatten)        (None, 493312)            0         
                                                                 
 dense_30 (Dense)            (None, 64)                31572032  
                                                                 
 dropout_30 (Dropout)        (None, 64)                0         
                                                          

In [62]:
y_pred=model.predict(X_test)
from numpy import argmax
yclass = argmax(y_pred,axis=1)
from sklearn.metrics import confusion_matrix
score = f1_score(y_test, yclass, average='macro')
print('F1 score:',score)
print(confusion_matrix(y_test['15420'].values, yclass))

F1 score: 0.46766712160960305
[[ 8 10  0 24]
 [ 3 20  0 15]
 [ 8  2  8  4]
 [12  9  2 60]]


We therefore see that the CNN model for disease classification has better performance than the ANN. We note, however, that the accuracy was still improving within the 10 epochs considered, so this could likely be improved. 

In this notebook, we have only considered a small selection of models, and further investigation would be warranted in order to improve the model:

*   Use all data records instead of the subset that was considered here for testing purposes.
*   While classification of diseases could have been higher, we should note that many of the records correspond to the same patient. Therefore, a final step in this work should be to average over all ECG records to give a final diagnosis.
*   Further exploration into the architecture of the neural networks and the parameters involved would be helpful.
* Enhanced investigation into the ECG records themselves: Should we weight features according to their importance and known results about distinguishing features? Other insights may be gained using domain knowledge.
*   Finally, the models should be analysed to establish which features were most important in classifying the record.

